# Solution `geopandas` & `cartopy`

Take the following code snippet as a starting point. It loads the NUTS regions of Europe, the power plant dataset, and a shapefile for the Danish [Natura2000](https://en.wikipedia.org/wiki/Natura_2000) natural protection areas.

```python
import cartopy
import pandas as pd
import geopandas as gpd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

url = "https://tubcloud.tu-berlin.de/s/RHZJrN8Dnfn26nr/download/NUTS_RG_10M_2021_4326.geojson"
nuts = gpd.read_file(url)

fn = "https://raw.githubusercontent.com/PyPSA/powerplantmatching/master/powerplants.csv"
df = pd.read_csv(fn, index_col=0)
geometry = gpd.points_from_xy(df["lon"], df["lat"])
ppl = gpd.GeoDataFrame(df, geometry=geometry, crs=4326)

url = "https://tubcloud.tu-berlin.de/s/mEpdmgBtmMbyjAr/download/Natura2000_end2021-DK.gpkg"
natura = gpd.read_file(url)
```

In [ ]:
import cartopy
import pandas as pd
import geopandas as gpd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

url = "https://tubcloud.tu-berlin.de/s/RHZJrN8Dnfn26nr/download/NUTS_RG_10M_2021_4326.geojson"
nuts = gpd.read_file(url)

fn = "https://raw.githubusercontent.com/PyPSA/powerplantmatching/master/powerplants.csv"
df = pd.read_csv(fn, index_col=0)
geometry = gpd.points_from_xy(df["lon"], df["lat"])
ppl = gpd.GeoDataFrame(df, geometry=geometry, crs=4326)

url = "https://tubcloud.tu-berlin.de/s/mEpdmgBtmMbyjAr/download/Natura2000_end2021-DK.gpkg"
natura = gpd.read_file(url)

**Task 1:** Identify the coordinate reference system of the `natura` GeoDataFrame.

In [ ]:
natura.crs

**Task 2:** Plot the `natura` GeoDataFrame on a map without transforming its CRS. Use `cartopy` for setting the projection of the figure and add coastlines and borders.

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.epsg(3035))
natura.plot(ax=ax)
ax.coastlines()
ax.add_feature(cartopy.feature.BORDERS);

**Task 3:** Identify the name of the largest protected area in the `natura` GeoDataFrame.

In [ ]:
i = natura.area.idxmax()
natura.loc[i, "SITENAME"]

**Task 4:** What is the total protection area in square kilometers.

In [ ]:
natura.dissolve().area.iloc[0] / 1e6

**Task 5:** The `natura` GeoDataFrame has a column `SITETYPE` that indicates the type of protected area. Calculate the total area for each type of protected area (again in square kilometers).

In [ ]:
natura.dissolve("SITETYPE").area / 1e6

**Task 6:** By how much (in percent) would the total area of protected areas increase if a buffer of 1 km around each protected area were also protected?

In [ ]:
original_area = natura.dissolve().area.div(1e6).iloc[0]

In [ ]:
expanded_area = gpd.GeoDataFrame(geometry=natura.buffer(1000)).dissolve().area.div(1e6).iloc[0]

In [ ]:
expanded_area / original_area * 100 - 100

**Task 7:** List the power plants that are located within protected areas. How many power plants are located within protected areas? Use the [`.sjoin()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html) function. Check the result by plotting these power plants on top of the protected areas.

In [ ]:
p = ppl.to_crs(natura.crs).sjoin(natura)
p

In [ ]:
len(p)

In [ ]:
ax = natura.plot()
p.plot(ax=ax, color="red", markersize=5);

**Task 8 (advanced):** What fraction of the natural protection area is located offshore? Use set operations with the [`.overlay()`](https://geopandas.org/en/stable/docs/user_guide/set_operations.html#set-operations-with-overlay) function and the NUTS regions `GeoDataFrame`.

:::{note}
Consult the [GeoPandas documentation](https://geopandas.org/en/stable/docs/user_guide/index.html) if you need an introduction to how to use the `.overlay()` function.
:::

In [ ]:
dk = nuts.query("LEVL_CODE == 0 and CNTR_CODE == 'DK'")

In [ ]:
overlay = gpd.overlay(natura, dk.to_crs(3035))

In [ ]:
onshore = overlay.dissolve().area.div(1e6).iloc[0]
total = natura.dissolve().area.div(1e6).iloc[0]


In [ ]:
onshore / total * 100